# Objective
The purpose is to create a demonstable prototype that mines purchase data and predicts categories similar to the input.
For elucidations sake we will divide the summary problem into sub problems.
*  Problem 1 -Given item A predict item B which is most associated through purchase patterns.


# Packages required
* Pandas for data frame
* Numpy for arrays

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

# Problem 1

# Import Data

In [2]:
data = pd.read_csv('../data/FMCGSales.csv', names = ['BillId','ItemId','ItemName','Level1','Level2','Level3','Level4','Level5','Level6'] )


In [3]:
#Dummy for pivot table
data['dummy'] = 1

In [4]:
data.head()

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
0,9,135291,DETTOL ANTISEPTIC LIQUID 500ML,FMCG,FMCG NON FOOD,OTC,HEALTH,FIRST Aid,ANTISEPTIC,1
1,14,152333,GULABARI 59ML,FMCG,FMCG NON FOOD,PERSONAL CARE,SKIN CARE,LOTION,BEAUTY & NOURISHMENT,1
2,14,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
3,16,203917,SAVLON ANTISEPTIC LIQUID 200ML,FMCG,FMCG NON FOOD,OTC,HEALTH,SOLUTION,ANTISEPTIC,1
4,21,203289,SAFI SYP 100ML,FMCG,AYUSH,UNANI,LIQUID & GELS,SYRUP,NaN,1


# manual

In [110]:
data.loc[data['ItemId']== 110828]

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
4181,22491,110828,ADIDAS DEO TEAM FORCE MEN 150ML,FMCG,FMCG NON FOOD,PERSONAL CARE,PERSONAL HYGIENE,FRAGRANCES,DEODORANT,1
6865,41148,110828,ADIDAS DEO TEAM FORCE MEN 150ML,FMCG,FMCG NON FOOD,PERSONAL CARE,PERSONAL HYGIENE,FRAGRANCES,DEODORANT,1
7749,48769,110828,ADIDAS DEO TEAM FORCE MEN 150ML,FMCG,FMCG NON FOOD,PERSONAL CARE,PERSONAL HYGIENE,FRAGRANCES,DEODORANT,1
9787,64410,110828,ADIDAS DEO TEAM FORCE MEN 150ML,FMCG,FMCG NON FOOD,PERSONAL CARE,PERSONAL HYGIENE,FRAGRANCES,DEODORANT,1


In [111]:
data.loc[data['ItemId']== 194831]

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
1882,10251,194831,PONDS MEN FACE WASH ENERGY CHARGE 100GM,FMCG,FMCG NON FOOD,PERSONAL CARE,SKIN CARE,FACE WASH,CLEANSING,1
6531,38944,194831,PONDS MEN FACE WASH ENERGY CHARGE 100GM,FMCG,FMCG NON FOOD,PERSONAL CARE,SKIN CARE,FACE WASH,CLEANSING,1
6866,41148,194831,PONDS MEN FACE WASH ENERGY CHARGE 100GM,FMCG,FMCG NON FOOD,PERSONAL CARE,SKIN CARE,FACE WASH,CLEANSING,1


# Data Exploration

In [5]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
BillId,10000.0,30108.5341,19185.501711,9.0,13684.00,27368.0,46706.75,65967.0
ItemId,10000.0,169527.2357,30982.008258,110810.0,144337.25,162672.0,200831.00,225105.0
dummy,10000.0,1.0000,0.000000,1.0,1.00,1.0,1.00,1.0


In [6]:
#is any row NULL ?
data.isnull().any().any(), data.shape

(True, (10000, 10))

In [7]:
#describe nullness
data.isnull().sum(axis=0)

BillId         0
ItemId         0
ItemName       0
Level1         0
Level2         0
Level3         0
Level4         0
Level5         5
Level6      1286
dummy          0
dtype: int64

Level 5 has 5 null values
Level 6 has 1286 null Values
Use data.dropna() to drop null rows if we are using that. Dont know if filling null values will solve the problem because the null values can have many values. Perhaps clustering can be employed to label these products first.

In [8]:
#data['pName'] = data['pName'].str.lower()

In [9]:
#data['pName'] = data['pName'].str.title()

In [10]:
#data['pName'] = data['pName'].str.replace(" ","")

In [11]:
#test for loc arbitrary item Id 220862
data.loc[data['ItemId'] == 220862]

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
2,14,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
148,848,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
403,2307,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
630,3638,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
638,3682,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
651,3730,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
704,4069,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
720,4152,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
724,4166,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1
851,4810,220862,VICKS VAPORUB 25GM,FMCG,FMCG NON FOOD,OTC,HEALTH,BALM,COLD,1


# Name Indexer

In [12]:
#new DF with itemName and itemID
names = data[['ItemName', 'ItemId']]
names.shape

(10000, 2)

In [13]:
names = names.drop_duplicates(subset = 'ItemName')
print(names.shape)
names.head()

(902, 2)


,ItemName,ItemId
0,DETTOL ANTISEPTIC LIQUID 500ML,135291
1,GULABARI 59ML,152333
2,VICKS VAPORUB 25GM,220862
3,SAVLON ANTISEPTIC LIQUID 200ML,203917
4,SAFI SYP 100ML,203289


In [14]:
# set index to pName
name_index = names.set_index('ItemName')
name_index.head()

,ItemId
ItemName,
DETTOL ANTISEPTIC LIQUID 500ML,135291
GULABARI 59ML,152333
VICKS VAPORUB 25GM,220862
SAVLON ANTISEPTIC LIQUID 200ML,203917
SAFI SYP 100ML,203289


In [15]:
#Sorting
name_index.sort_values('ItemName')

,ItemId
ItemName,
110PELLETS SUGAR FREE GOLD,209917
ADIDAS DEO DYNAMIC PULSE 150ML,110810
ADIDAS DEO FRUITY RHYTHM 150ML,110815
ADIDAS DEO ICE DIVE MEN 150ML,110819
ADIDAS DEO PURE GAME MEN 150ML,110824
ADIDAS DEO TEAM FORCE MEN 150ML,110828
ADIDAS DEO VICTORY LEG MEN 150ML,110829
AIR WICK AQUA MIST SPRAY 345ML,111340
ALL OUT BEDTIME REFILL 45ML,112012


In [16]:
name_index.loc['WOODWARDS GRIPE WATER 200ML']

ItemId    223749
Name: WOODWARDS GRIPE WATER 200ML, dtype: int64

# Items Index

In [17]:
normal_index = name_index.reset_index()
normal_index.head()

,ItemName,ItemId
0,DETTOL ANTISEPTIC LIQUID 500ML,135291
1,GULABARI 59ML,152333
2,VICKS VAPORUB 25GM,220862
3,SAVLON ANTISEPTIC LIQUID 200ML,203917
4,SAFI SYP 100ML,203289


In [18]:
items_index =  normal_index.set_index('ItemId')
items_index.head()

,ItemName
ItemId,
135291,DETTOL ANTISEPTIC LIQUID 500ML
152333,GULABARI 59ML
220862,VICKS VAPORUB 25GM
203917,SAVLON ANTISEPTIC LIQUID 200ML
203289,SAFI SYP 100ML


In [19]:
items_index.index.is_unique

True

In [20]:
#itemIndexTest.loc[88510]

In [21]:
#itemIndexTest.loc[88510]

In [22]:
name_index.index.is_unique

True

# Pivot Table

In [23]:
matrix = data.pivot_table(values='dummy',index ='BillId', columns ='ItemId')
matrix.head()

ItemId,110810,110815,110819,110824,110828,110829,111340,112012,112019,112031,...,223563,223584,223748,223749,224914,224915,224916,225093,225097,225105
BillId,,,,,,,,,,,,,,,,,,,,,
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
matrix.shape

(7485, 904)

In [25]:
matrix_dummy = matrix.copy().fillna(0)

In [26]:
matrix_dummy.head()

ItemId,110810,110815,110819,110824,110828,110829,111340,112012,112019,112031,...,223563,223584,223748,223749,224914,224915,224916,225093,225097,225105
BillId,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Skewed Jaccardian
We define a jaccardian as intersection over union. A skewed Jaccardian is intersection over set A

### method from http://na-o-ys.github.io/others/2015-11-07-sparse-vector-similarities.html

In [38]:
#needs parameter in scipy.sparse.csc_matrix
type(matrix_dummy)

pandas.core.frame.DataFrame

# Experiment

In [65]:
sparse_matrix = sparse.csc_matrix(matrix_dummy)

In [66]:
type(sparse_matrix)

scipy.sparse.csc.csc_matrix

In [67]:
def jaccard_similarities(mat):
    cols_sum = mat.getnnz(axis=0)
    ab = mat.T * mat

    # for rows
    aa = np.repeat(cols_sum, ab.getnnz(axis=0))
    # for columns
    bb = cols_sum[ab.indices]

    similarities = ab.copy()
    similarities.data /= (aa + bb - ab.data)

    return similarities

In [68]:
jaccard_similarities =  jaccard_similarities(sparse_matrix)

In [76]:
jaccard_similarities

<904x904 sparse matrix of type '<class 'numpy.float64'>'
	with 8420 stored elements in Compressed Sparse Row format>

In [94]:
print(jaccard_similarities)

  (0, 351)	0.030303030303030304
  (0, 768)	0.0032258064516129032
  (0, 636)	0.041666666666666664
  (0, 596)	0.1
  (0, 320)	0.08333333333333333
  (0, 0)	1.0
  (1, 252)	0.021739130434782608
  (1, 1)	1.0
  (2, 442)	0.14285714285714285
  (2, 598)	0.14285714285714285
  (2, 714)	0.04
  (2, 345)	0.01
  (2, 2)	1.0
  (3, 67)	0.043478260869565216
  (3, 3)	1.0
  (4, 717)	0.018867924528301886
  (4, 780)	0.010752688172043012
  (4, 745)	0.16666666666666666
  (4, 4)	1.0
  (5, 884)	0.045454545454545456
  (5, 306)	0.08333333333333333
  (5, 176)	0.005128205128205128
  (5, 5)	1.0
  (6, 640)	0.16666666666666666
  (6, 6)	1.0
  :	:
  (899, 448)	0.05263157894736842
  (899, 37)	0.030303030303030304
  (899, 872)	0.011764705882352941
  (899, 14)	0.1
  (899, 899)	1.0
  (900, 900)	1.0
  (900, 872)	0.011235955056179775
  (901, 377)	0.005649717514124294
  (901, 375)	0.03571428571428571
  (901, 873)	0.024390243902439025
  (901, 780)	0.009523809523809525
  (901, 269)	0.05555555555555555
  (901, 901)	1.0
  (901, 869)	

In [79]:
matrix.columns

Int64Index([110810, 110815, 110819, 110824, 110828, 110829, 111340, 112012,
            112019, 112031,
            ...
            223563, 223584, 223748, 223749, 224914, 224915, 224916, 225093,
            225097, 225105],
           dtype='int64', name='ItemId', length=904)

In [97]:
jaccardian = pd.DataFrame(jaccard_similarities.toarray(), index = matrix.columns,columns = matrix.columns)

In [98]:
jaccardian.head()

ItemId,110810,110815,110819,110824,110828,110829,111340,112012,112019,112031,...,223563,223584,223748,223749,224914,224915,224916,225093,225097,225105
ItemId,,,,,,,,,,,,,,,,,,,,,
110810,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110815,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110819,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110824,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110828,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# End Experiment

# Final Function

In [105]:
# Final Function
# Takes title, userId and returns to 25 sorted movie names on est
def reco(idx):

    similar_itemids = jaccardian.loc[idx].to_frame(name=None)
    topItemIds = similar_itemids.sort_values(by =[idx],ascending = False).head(26)
    topItemNames = topItemIds.merge(normal_index, how ='left', on = 'ItemId' )

    
    return topItemNames.head(10)



In [106]:
reco(110828)

,ItemId,110828,ItemName
0,110828,1.000000,ADIDAS DEO TEAM FORCE MEN 150ML
1,194831,0.166667,PONDS MEN FACE WASH ENERGY CHARGE 100GM
2,191624,0.018868,PATANJALI DANT KANTI DENTAL TG FG 200GM
3,205877,0.010753,SERRATA POMACE OLIVE OIL 1LTR
4,110810,0.000000,ADIDAS DEO DYNAMIC PULSE 150ML
5,182850,0.000000,NIVEA DEO WHITENING SENSITIVE 150ML
6,182816,0.000000,NIVEA DEO DEODRIZER SPRINT 120ML
7,182821,0.000000,NIVEA DEO FRESH ACTIV ORIG 150ML
8,182825,0.000000,NIVEA DEO FRESH CITRUS&CARE 120ML
9,182826,0.000000,NIVEA DEO FRESH COMFORT 150ML


In [107]:
reco(203289)

,ItemId,203289,ItemName
0,203289,1.000000,SAFI SYP 100ML
1,210589,0.047619,SUNSILK SHAMPOO SOFT SMOOTH YELLOW 675ML
2,148347,0.041667,GARNIER FACE WASH PURE ACTIVE NEEM 50GM
3,210568,0.040000,SUNSILK SHAMPOO LONG & HEALTHY GROWTH 340ML
4,205666,0.038462,SENSODYNE PASTE DEEP CLEAN 70GM
5,130309,0.037037,COLGATE PASTE TOTAL CHARCOAL D 140GM
6,182683,0.035714,NITYAM CHURAN 50GM
7,133501,0.033333,DABUR GLUCOSE REG VITD 125GM
8,185164,0.025641,ODOMOS NATURAL CREAM 50GM
9,220774,0.023256,VICCO TURMERIC CREAM 50GM


In [108]:
reco(220862)

,ItemId,220862,ItemName
0,220862,1.000000,VICKS VAPORUB 25GM
1,220844,0.045455,VICKS INHALLER
2,133581,0.036697,DABUR PUDIN HARA 10ML
3,113364,0.022472,AMRUTANJAN BALM STRONG GREEN 55ML
4,122054,0.021739,BOROLINE CREAM 20GM
5,177551,0.019802,MOOV KNEE & JOINT PAIN RELIFE ORTHO 15GM
6,203833,0.015707,SAT ISABGOL 200GM
7,191626,0.014388,PATANJALI DANT KANTI FAIMLY PACK 300GM
8,172218,0.011765,MANFORCE CONDOM LITCHI FLAVOURED 3PCS
9,224915,0.011765,YOGI KANTHIKA TAB 20s


In [ ]:
reco(16288)

In [ ]:
reco(16722)

In [ ]:
reco(111852)

In [ ]:
reco(86616)